# Find Latitude and Longitude of GOES points

- This outputs a file that save the lat and lon of all the GOES step points

## Calculate latitude and longitude from GOES ABI fixed grid projection data
GOES ABI fixed grid projection is a map projection relative to the GOES satellite  
- Units: latitude in °N (°S < 0), longitude in °E (°W < 0)  

See GOES-R Product User Guide (PUG) Volume 5 (L2 products) Section 4.2.8 for details & example of calculations "file_id" is an ABI L1b or L2 .nc file opened using the netCDF4 library code from https://www.star.nesdis.noaa.gov/atmospheric-composition-training/python_abi_lat_lon.php  

- Acknowledgement:  NOAA/NESDIS/STAR Aerosols and Atmospheric Composition Science Team  
- Their code is written for numpy arrays not xarray, so I updated it to work with xarray datasets  

- list of GOES products https://github.com/blaylockbk/goes2go/blob/main/goes2go/product_table.txt


In [ ]:
######################### INPUTS #############################

files_fname = './../goes_filenames_test_20231206-20231207.csv'

#import configuration location and filepath
from myconfig import *

# Target latitude and longitude MIAMI
#target_name = 'MilwaukeeTrench'
#target_lat = 19.71361111111111  # Example: Latitude of Milwaukee Trench
#target_lon = -67.31083333333333  # Example: Longitude of Milwaukee Trench

#############################################################


In [ ]:
from goes2go import GOES
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings
import numpy as np
warnings.filterwarnings('ignore')
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pandas as pd
import s3fs
import xarray as xr
import os

def calculate_degrees(file_id):
    # Read in GOES ABI fixed grid projection variables and constants
    x_coordinate_1d = file_id.variables['x'][:]  # E/W scanning angle in radians
    y_coordinate_1d = file_id.variables['y'][:]  # N/S elevation angle in radians
    projection_info = file_id.variables['goes_imager_projection']
    lon_origin = projection_info.attrs.get('longitude_of_projection_origin')
    H = projection_info.attrs.get('perspective_point_height')+projection_info.attrs.get('semi_major_axis')
    r_eq = projection_info.attrs.get('semi_major_axis')
    r_pol = projection_info.attrs.get('semi_minor_axis')
    
    # Create 2D coordinate matrices from 1D coordinate vectors
    x_coordinate_2d, y_coordinate_2d = np.meshgrid(x_coordinate_1d, y_coordinate_1d)
    
    # Equations to calculate latitude and longitude
    lambda_0 = (lon_origin*np.pi)/180.0  
    a_var = np.power(np.sin(x_coordinate_2d),2.0) + (np.power(np.cos(x_coordinate_2d),2.0)*(np.power(np.cos(y_coordinate_2d),2.0)+(((r_eq*r_eq)/(r_pol*r_pol))*np.power(np.sin(y_coordinate_2d),2.0))))
    b_var = -2.0*H*np.cos(x_coordinate_2d)*np.cos(y_coordinate_2d)
    c_var = (H**2.0)-(r_eq**2.0)
    r_s = (-1.0*b_var - np.sqrt((b_var**2)-(4.0*a_var*c_var)))/(2.0*a_var)
    s_x = r_s*np.cos(x_coordinate_2d)*np.cos(y_coordinate_2d)
    s_y = - r_s*np.sin(x_coordinate_2d)
    s_z = r_s*np.cos(x_coordinate_2d)*np.sin(y_coordinate_2d)
    
    # Ignore numpy errors for sqrt of negative number; occurs for GOES-16 ABI CONUS sector data
    np.seterr(all='ignore')
    
    abi_lat = (180.0/np.pi)*(np.arctan(((r_eq*r_eq)/(r_pol*r_pol))*((s_z/np.sqrt(((H-s_x)*(H-s_x))+(s_y*s_y))))))
    abi_lon = (lambda_0 - np.arctan(s_y/(H-s_x)))*(180.0/np.pi)
    
    return abi_lat, abi_lon

def forward_fill_2d(arr):
    # Loop through each column
    for i in range(arr.shape[1]):
        mask = np.isnan(arr[:, i])
        # Forward fill NaN values
        arr[mask, i] = np.interp(np.flatnonzero(mask), np.flatnonzero(~mask), arr[~mask, i])
    return arr

def find_nearest_indices(lat_arr, lon_arr, target_lat, target_lon):
    # Find the nearest latitude index
    lat_idx = (np.abs(lat_arr - target_lat)).argmin()
    # Find the nearest longitude index
    lon_idx = (np.abs(lon_arr - target_lon)).argmin()
    return lat_idx, lon_idx

def calculate_points(istep,lon_idx,lat_idx):
    #how big do we want to have the box?
    #istep is how many grid points away from the center that we want to go   
    # List of points you want to subset around point x
    #   *  *  *
    #   *  x  *
    #   *  *  *
    #north_point = [lat_idx+istep,lon_idx]
    #east_point = [lat_idx,lon_idx+istep]
    #south_point = [lat_idx-istep,lon_idx]
    #west_point = [lat_idx,lon_idx-istep]
    #northeast_point = [lat_idx+istep,lon_idx+istep]
    #northwest_point = [lat_idx+istep,lon_idx-istep]
    #southeast_point = [lat_idx-istep,lon_idx+istep]
    #southwest_point = [lat_idx-istep,lon_idx-istep]
    points = [
        {"i": int(lon_idx), "j": int(lat_idx)+istep, "name": 'N'},
        {"i": int(lon_idx)+istep, "j": int(lat_idx)+istep, "name": 'NE'},
        {"i": int(lon_idx)+istep, "j": int(lat_idx), "name": 'East'},
        {"i": int(lon_idx)+istep, "j": int(lat_idx)-istep, "name": 'SE'},
        {"i": int(lon_idx), "j": int(lat_idx)-istep, "name": 'S'},
        {"i": int(lon_idx)-istep, "j": int(lat_idx)-istep, "name": 'SW'},
        {"i": int(lon_idx)-istep, "j": int(lat_idx), "name": 'W'},
        {"i": int(lon_idx)-istep, "j": int(lat_idx)+istep, "name": 'NW'},
    ]
    return points

def get_start_end_time(fname):
    #goes filenames structure https://geonetcast.wordpress.com/2017/04/27/goes-16-file-naming-convention/
    #use filename to find start/end times for data
    tem = str(fname).split('/')
    tem2,i = tem[5],25
    dt_start = datetime.strptime(tem2[i:i+13], '%Y%j%h%M%S')
    tem2,i = tem[5],41
    dt_end = datetime.strptime(tem2[i:i+13], '%Y%j%h%M%S')
    return dt_start,dt_end

def already_read(fname,gfile):
    start_time,end_time = get_start_end_time(fname)
    isum = gfile.read_value.loc[start_time:end_time].sum().data
    if isum>0:
        return True
    else:
        return False


In [ ]:
fs = s3fs.S3FileSystem(anon=True) #connect to s3 bucket!
df = pd.read_csv(files_fname)
file_location=df.file
file_ob = [fs.open('s3://'+file) for file in file_location]        #open connection to files
print(file_ob[0])
fname = file_ob[0]
ds = xr.open_dataset(file_ob[0]) #note file is super messed up formatting
abi_lat, abi_lon = calculate_degrees(ds)
abi_lat = forward_fill_2d(abi_lat.copy())
abi_lon = forward_fill_2d(abi_lon.copy())

In [ ]:
# calculate lat and lon from points

# Find nearest indices
lat_idx, lon_idx = find_nearest_indices(abi_lat[:,0], abi_lon[0,:], target_lat, target_lon)
lat_idx, lon_idx = find_nearest_indices(abi_lat[:,lon_idx], abi_lon[lat_idx,:], target_lat, target_lon)
        
#create map of lat/lon

step = np.arange(1,20)  # 2 steps
point = np.arange(0,8)  # 3 points
lat_data = np.zeros((19,8))
lon_data = np.zeros((19,8))
# Create the xarray dataset
ds_loc = xr.Dataset({"lat": (("step", "point"), lat_data),"lon": (("step", "point"),lon_data),}, coords={  "step": step,  "point": point    })

abi_lat_tem = abi_lat[(lat_idx-25):(lat_idx+25),(lon_idx-25):(lon_idx+25)]
abi_lon_tem = abi_lon[(lat_idx-25):(lat_idx+25),(lon_idx-25):(lon_idx+25)]
for istep in range(1,20):
    points = calculate_points(istep,25,25) #lon_idx,lat_idx)
    for p in range(len(points)):
        ds_loc["lat"].loc[{"step": istep, "point": p}] = abi_lat_tem[points[p].get('j'),points[p].get('i')]
        ds_loc["lon"].loc[{"step": istep, "point": p}] = abi_lon_tem[points[p].get('j'),points[p].get('i')]

nc_fname = output_path+'goes_timeseries_point_latlon.nc'
csv_fname = output_path+'goes_timeseries_points_latlon.csv'
ds_loc.to_netcdf(nc_fname) 
df = ds_loc.to_dataframe()
df.to_csv(csv_fname)

